<a href="https://colab.research.google.com/github/abSETEd/check_phone/blob/abSETEd-patch-1/googlemapsver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas requests bs4
!pip install quote
!pip install pyexcel-ods3


import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import quote
import time
from google.colab import files
import re
import io
import pyexcel_ods3

def carregar_planilha():
  print("Faça upload da sua planilha:")
  upload = files.upload()
  nome_aquivo = list(upload.keys()) [0]
  try:
    if nome_arquivo.endswith('.csv'):
      df = pd.read_csv(io.BytesIO(arquivo_carregado))
    elif nome_arquivo.endswith('.xlsx'):
      df = pd.read_escel(io.BytesIO(arquivo_carregado), engine='openpyxl')
    elif nome_arquivo.endswith('.ods'):
      ods_data = pyexcel_ods3.get_data(io.BytesIO(arquivo_carregado))
      df = pd.DataGrame(ods_data[list(ods_data.keys())[0]][1:], columns=ods_data[list(ods_data.keys())[0]][0])
    else:
      raise ValueError("Formato de arquivo não suportado")

     if "Site" not in df.columns:
      raise ValueError("A Planilha precisa ter uma coluna chamada 'Site'")
    return df["Site"].tolist()
  except Exception as e:
    print(f"Erro ao carregar a planilha: {e}")
    return[]

def pesquisar_google_maps(site):
  query = quote(site)
  url = f"https://www.google.com/maps/search/{query}"
  return url

def pesquisar_linkedin(site):
  query = quote(site)
  url = f"https://www.linkedin.com/search/results/all/?keywords={query}"
  return url

def extrair_dados(url):
  try:
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    dados= {
        'nome': '',
        'telefone': '',
        'endereco': '',
        'email': ''
    }

    for tag in soup.find_all(['h1', 'h2', 'h3']):
      if not dados['nome']:
        dados['nome'] = tag.strip()

    for tag in soup.find_all(string=re.compile(r'\(\d{3}\)\s*\d{3}-\d{4}')):
      if not dados['telefone']:
        dados['telefone'] = tag.strip()

    for tag in soup.find_all(string=re.compile(r'@[\w-]+\.com')):
      if not dados['email']:
        dados['email'] = tag.strip()

    return dados
  except Exception as e:
    print(f"Erro ao extrair dados: {e}")
    return {'nome': '', 'telefone': '', 'endereco': '', 'email': ''}

def main():
  sites = carregar_planilha()

  if not sites:
    print("Nenhum site encontrado para pesquisa.")
    return

    resultados = []

    with open('resultados_pesquisa.csv', 'w', newline='', encoding='utf-8') as csvfile:
      dieldnames = ['Site', 'Nome', 'Telefone', 'Endereço', 'Email', 'URLS Pesquisada']
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
      writer.writeheader()

      for site in sites:
        print(f"Pesquisando:{site}")

        urls_pesquisa = pesquisar_google_maps(site)

        dados = extrair_dados(url_pesquisa)

        resultado = {
            'Site': site,
            'Nome': dados['nome'],
            'Telefone': dados['telefone'],
            'Endereço': dados['endereco'],
            'Email': dados['email'],
            'URL Pesquisada': url_pesquisa
        }
        resultados.append(resultado)
        writer.writerow(resultado)

        time.sleep(5)

    print("Pesquisa concluida! Resultados salvos em 'resultados_pesquisa.csv'")
    files.download('resultados_pesquisa.csv')

main()

Faça upload da sua planilha:


Saving 25.03.24 teste (1)_com_telefones (1).ods to 25.03.24 teste (1)_com_telefones (1) (1).ods
Erro ao carregar a planilha: name 'nome_arquivo' is not defined
Nenhum site encontrado para pesquisa.
